# Data Wrangling in Python

## Import Libraries

In [32]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px

## Table Variables - Reference

In [33]:
#Annex table = annex
#Notes table = notes_dict (# it is a dictionary)

#Cleaned Table 1 = table1_clean
#Cleaned Table 2 = table2_clean
#Cleaned Table 3 = table3_clean
#Cleaned Table 4 = table4_clean
#Cleaned Table 5 = table5_clean
#Cleaned Table 6 = table6_MyStock
#Cleaned Table 6 Refugee as a percentage of international migrant stock = table6_RP
#Cleaned Table 6 Annual Rate of Change of Refugee Stock = table6_AR

# Part I: Data Cleaning

## Read Notes Table from Github

In [34]:
notes = pd.read_excel(r"https://github.com/teresalau/inf1340-programmingfordatascience-fa22/raw/main/project%20data/UN_MigrantStockTotal_2015.xlsx", sheet_name = "NOTES")

#Clean up the Notes Table

notes = notes.iloc[14:]
notes.rename(columns = {
    "Unnamed: 0": "Notes Label",
    "Unnamed: 1": "Notes Content" 
}, inplace = True)

# Convert the Notes table to a dictionary

notes_dict = notes.set_index("Notes Label").T.to_dict("list")

## Read Annex Table from Github

In [123]:
annex1 = pd.read_excel(r"https://github.com/teresalau/inf1340-programmingfordatascience-fa22/raw/main/project%20data/UN_MigrantStockTotal_2015.xlsx", sheet_name = "ANNEX")

#Clean up the annex table
#Remove top 12 rows as they are empty

annex = annex1.iloc[14:]
annex.columns = ["Country code",
                 "Country or Area",
                 "Country Sort Order",
                 "Major Area",
                 "Major Area Code",
                 "Major Area Sort Order",
                 "Region",
                 "Region Code",
                 "Region Sort Order",
                 "Developed Region",
                 "Least Developed Country",
                 "Sub-Saharan Africa"
                ]

#reset index as country codes to perform future inner join operation with other tables
annex = annex.astype({"Country code": "int64"}).set_index("Country code")

annex.head(10)

,Country or Area,Country Sort Order,Major Area,Major Area Code,Major Area Sort Order,Region,Region Code,Region Sort Order,Developed Region,Least Developed Country,Sub-Saharan Africa
Country code,,,,,,,,,,,
4,Afghanistan,99,Asia,935,71,Southern Asia,5501,98,No,Yes,No
8,Albania,154,Europe,908,127,Southern Europe,925,153,Yes,No,No
12,Algeria,40,Africa,903,7,Northern Africa,912,39,No,No,No
16,American Samoa,257,Oceania,909,238,Polynesia,957,256,No,No,No
20,Andorra,155,Europe,908,127,Southern Europe,925,153,Yes,No,No
24,Angola,30,Africa,903,7,Middle Africa,911,29,No,Yes,Yes
660,Anguilla,182,Latin America and the Caribbean,904,180,Caribbean,915,181,No,No,No
28,Antigua and Barbuda,183,Latin America and the Caribbean,904,180,Caribbean,915,181,No,No,No
32,Argentina,218,Latin America and the Caribbean,904,180,South America,931,217,No,No,No


## Read Excel Workbook from Github - Table 1

In [37]:
table1 = pd.read_excel(r"https://github.com/teresalau/inf1340-programmingfordatascience-fa22/raw/main/project%20data/UN_MigrantStockTotal_2015.xlsx", sheet_name = "Table 1")

### Remove blank rows and rename columns (Table 1)

In [38]:
#remove top 12 row as they are empty
table1 = table1.iloc[13:]

#Preference Setting - display more columns
pd.set_option("display.max_columns", 25)

#Rename Columns
table1.rename(columns = 
                    {"Unnamed: 0":"Sort/Order", 
                     "Unnamed: 1": "Major Area/Region/Country",
                     "Unnamed: 2":"Notes",
                     "Unnamed: 3": "Country Code",
                     "Unnamed: 4": "Type of Data",
                     "Unnamed: 5": "Both Sexes_1990",
                     "Unnamed: 6": "Both Sexes_1995",
                     "Unnamed: 7": "Both Sexes_2000",
                     "Unnamed: 8": "Both Sexes_2005",
                     "Unnamed: 9":"Both Sexes_2010",
                     "Unnamed: 10":"Both Sexes_2015",
                     "Unnamed: 11":"M_1990",
                     "Unnamed: 12": "M_1995",
                     "Unnamed: 13": "M_2000",
                     "Unnamed: 14": "M_2005",
                     "Unnamed: 15":"M_2010",
                     "Unnamed: 16": "M_2015", 
                     "Unnamed: 17": "F_1990",
                     "Unnamed: 18": "F_1995",
                     "Unnamed: 19": "F_2000", 
                     "Unnamed: 20": "F_2005",
                     "Unnamed: 21":"F_2010",
                     "Unnamed: 22":"F_2015"}, inplace = True)

#remove rows containing old column headers
table1_removetop2 = table1.iloc[2:]

### Unpivot Columns containing Information Year & Gender (Table 1)

In [39]:
#Unpivot columns containing sex and year to rows

melt_table1 = pd.melt(table1_removetop2, id_vars = ["Sort/Order", "Major Area/Region/Country","Notes","Country Code","Type of Data"],
                     var_name = "Sex_Year", value_name = "Migrant Stock")

#split gender year by delimiter
melt_table1[["Sex", "Year"]] = melt_table1["Sex_Year"].str.split("_", expand = True)


#Remove Gender_Year column

table1_dropgy = melt_table1.drop(["Sex_Year"], axis = 1)

#Recode "M" and "F" as proper labels "Male" and "Female"
table1_dropgy = table1_dropgy.replace({"M":"Male","F":"Female"})

### Clean "Type of Data" column (Table 1)

In [40]:
#Check for unique value in "Type of Data" column
table1_dropgy["Type of Data"].unique()

#Clean up the inconsistent labels in "Type of Data"
table1_dropgy["Type of Data"] = table1_dropgy["Type of Data"].str.replace('B R ','B R')

#Build a dictionary to label the different types of population data incoporated in the data

data_type_dict = {
    "B":"Foreign-born Population",
    "R":"Refugee Population Included in Estimate of International Migrants",
    "I":"No Data on International Migrant",
    "C":"Foreign Citizen",
    "na":"No Additional Information"
}

#Replace nan with na in column "Type of Data" as it leads to error in the data replacement process

table1_dropgy["Type of Data"] = table1_dropgy["Type of Data"].replace(to_replace = [np.nan], value = ["na"])

#Replacing values in "Type of Data" with the descriptions in the dictionary

table1_dropgy["Type of Data"] = table1_dropgy["Type of Data"].apply(lambda x: "; ".join([data_type_dict[i] for i in x.strip().split()]))

### Remove regions/major areas from rows using inner join (Table 1)

In [41]:
# Inner Join Table 1 and Annex Table to get rid of regions and continents names from country row
# The Annex table has the breakdown of country, region and major area that I can reference

table1_joined = table1_dropgy.join(annex, on = "Country Code", how = "inner")

### Re-order Table 1 following the Inner Join (Table 1)

In [42]:
#Drop duplicate columns as a result of the inner join with the annex table & reset index
table1_joined = table1_joined.drop(columns = ["Sort/Order",
                                             "Major Area/Region/Country",
                                             ]).reset_index(drop = True)

#Rearrange table 1 to improve readibility
table1_clean = table1_joined[[
    'Major Area Sort Order',
    'Major Area Code',
    'Major Area',
    'Region Sort Order',
    'Region Code',
    'Region',
    'Country Sort Order',
    'Country Code',
    'Country or Area',
    'Year',
    'Sex',
    'Migrant Stock',
    'Type of Data',
    'Developed Region',
    'Least Developed Country',
    'Sub-Saharan Africa',
    'Notes'
]]

#Change data type for Migrant Stock from object to float
table1_clean["Migrant Stock"] = table1_clean["Migrant Stock"].replace(to_replace = [".."], value = [np.nan])
table1_clean.astype({"Migrant Stock": "float64"})

#Display cleaned Table 1
table1_clean

,Major Area Sort Order,Major Area Code,Major Area,Region Sort Order,Region Code,Region,Country Sort Order,Country Code,Country or Area,Year,Sex,Migrant Stock,Type of Data,Developed Region,Least Developed Country,Sub-Saharan Africa,Notes
0,7,903,Africa,8,910,Eastern Africa,9,108,Burundi,1990,Both Sexes,333110.0,Foreign-born Population; Refugee Population In...,No,Yes,Yes,NaN
1,7,903,Africa,8,910,Eastern Africa,9,108,Burundi,1995,Both Sexes,254853.0,Foreign-born Population; Refugee Population In...,No,Yes,Yes,NaN
2,7,903,Africa,8,910,Eastern Africa,9,108,Burundi,2000,Both Sexes,125628.0,Foreign-born Population; Refugee Population In...,No,Yes,Yes,NaN
3,7,903,Africa,8,910,Eastern Africa,9,108,Burundi,2005,Both Sexes,172874.0,Foreign-born Population; Refugee Population In...,No,Yes,Yes,NaN
4,7,903,Africa,8,910,Eastern Africa,9,108,Burundi,2010,Both Sexes,235259.0,Foreign-born Population; Refugee Population In...,No,Yes,Yes,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4171,238,909,Oceania,256,957,Polynesia,265,876,Wallis and Futuna Islands,1995,Female,821.0,Foreign-born Population,No,No,No,NaN
4172,238,909,Oceania,256,957,Polynesia,265,876,Wallis and Futuna Islands,2000,Female,997.0,Foreign-born Population,No,No,No,NaN
4173,238,909,Oceania,256,957,Polynesia,265,876,Wallis and Futuna Islands,2005,Female,1171.0,Foreign-born Population,No,No,No,NaN
4174,238,909,Oceania,256,957,Polynesia,265,876,Wallis and Futuna Islands,2010,Female,1375.0,Foreign-born Population,No,No,No,NaN


## Read Table 2 from Excel Workbook on Github

In [43]:
table2 = pd.read_excel(r"https://github.com/teresalau/inf1340-programmingfordatascience-fa22/raw/main/project%20data/UN_MigrantStockTotal_2015.xlsx", sheet_name = "Table 2")

### Remove Blank Rows & Rename Column Names (Table 2)

In [44]:
#Remove top 14 blank rows
table2_removetop14 = table2.iloc[15:].copy()

#Rename columns with proper columns headers
table2_removetop14.rename(columns = {
    "Unnamed: 0":"Sort\norder",
    "Unnamed: 1":"Major Area/Region/Country",
    "Unnamed: 2":"Notes",
    "Unnamed: 3":"Country Code",
    "Unnamed: 4": "Both Sexes_1990",
    "Unnamed: 5": "Both Sexes_1995",
    "Unnamed: 6": "Both Sexes_2000",
    "Unnamed: 7": "Both Sexes_2005",
    "Unnamed: 8": "Both Sexes_2010",
    "Unnamed: 9": "Both Sexes_2015",
    "Unnamed: 10":"M_1990",
    "Unnamed: 11":"M_1995",
    "Unnamed: 12":"M_2000",
    "Unnamed: 13":"M_2005",
    "Unnamed: 14":"M_2010",
    "Unnamed: 15":"M_2015",
    "Unnamed: 16":"F_1990",
    "Unnamed: 17":"F_1995",
    "Unnamed: 18":"F_2000",
    "Unnamed: 19":"F_2005",
    "Unnamed: 20":"F_2010",
    "Unnamed: 21":"F_2015"
}, inplace = True)

### Create Seperate Gender & Year columns (Table 2)

In [45]:
#Unpivot Gender_Year column to rows

table2_MFflat = pd.melt(table2_removetop14, id_vars =[
    "Sort\norder",
    "Major Area/Region/Country",
    "Notes",
    "Country Code"], var_name = "Sex_Year", 
    value_name = "Total Population")

#seperate the Gender_Year columns by delimiter
table2_MFflat[["Sex", "Year"]] = table2_MFflat["Sex_Year"].str.split("_", expand = True)

#Remove Gender_Year column
table2_MFflat = table2_MFflat.drop(columns=["Sex_Year"])
table2_MFflat

#Rename "M" and "F" to proper labels as "Male" and "Female". "Both Sexes" labels remain unchanged.
table2_MFflat = table2_MFflat.replace({
    "M":"Male",
    "F":"Female"
})


### Remove world/region/major area information by inner join (join with Annex table) (Table 2)

In [46]:
table2_joined = table2_MFflat.join(annex, on = "Country Code", how = "inner")

### Clean up following the Inner Join & Rearrange columns order (Table 2)

In [47]:
#Drop duplicate columns as a result of the inner join with the annex table & reset index
table2_joined = table2_joined.drop(columns = ["Sort\norder",
                                             "Major Area/Region/Country",
                                             ]).reset_index(drop = True)

#Rearrange columns to improve readability
table2_clean = table2_joined[[
    'Major Area Sort Order',
    'Major Area Code',
    'Major Area',
    'Region Sort Order',
    'Region Code',
    'Region',
    'Country Sort Order',
    'Country Code',
    'Country or Area',
    'Year',
    'Sex',
    'Total Population',
    'Developed Region',
    'Least Developed Country',
    'Sub-Saharan Africa',
    'Notes'
]]

#Replace the ".." with nan under the total population column
table2_clean["Total Population"] = table2_clean["Total Population"].replace(to_replace = [".."], value = [np.nan])

#The values are expressed as x1000 in the original dataset. Converting the value in full to avoid confustion. 
table2_clean["Total Population"] = table2_clean["Total Population"]*1000

#Convert data type for "Total Population" column to float
table2_clean.astype({"Total Population": "float64"})

#Display cleaned table 2
table2_clean

,Major Area Sort Order,Major Area Code,Major Area,Region Sort Order,Region Code,Region,Country Sort Order,Country Code,Country or Area,Year,Sex,Total Population,Developed Region,Least Developed Country,Sub-Saharan Africa,Notes
0,7,903,Africa,8,910,Eastern Africa,9,108,Burundi,1990,Both Sexes,5613141.0,No,Yes,Yes,NaN
1,7,903,Africa,8,910,Eastern Africa,9,108,Burundi,1995,Both Sexes,6239030.0,No,Yes,Yes,NaN
2,7,903,Africa,8,910,Eastern Africa,9,108,Burundi,2000,Both Sexes,6767073.0,No,Yes,Yes,NaN
3,7,903,Africa,8,910,Eastern Africa,9,108,Burundi,2005,Both Sexes,7934213.0,No,Yes,Yes,NaN
4,7,903,Africa,8,910,Eastern Africa,9,108,Burundi,2010,Both Sexes,9461117.0,No,Yes,Yes,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4171,238,909,Oceania,256,957,Polynesia,265,876,Wallis and Futuna Islands,1995,Female,NaN,No,No,No,NaN
4172,238,909,Oceania,256,957,Polynesia,265,876,Wallis and Futuna Islands,2000,Female,NaN,No,No,No,NaN
4173,238,909,Oceania,256,957,Polynesia,265,876,Wallis and Futuna Islands,2005,Female,NaN,No,No,No,NaN
4174,238,909,Oceania,256,957,Polynesia,265,876,Wallis and Futuna Islands,2010,Female,NaN,No,No,No,NaN


## Import Table 3 from Github

In [48]:
table3 = pd.read_excel(r"https://github.com/teresalau/inf1340-programmingfordatascience-fa22/raw/main/project%20data/UN_MigrantStockTotal_2015.xlsx", sheet_name = "Table 3")

### Remove Blank Rows and Rename Columns (Table 3)

In [49]:
#Rename Columns
table3.rename(columns = {
    "Unnamed: 0":"Sort/Order",
    "Unnamed: 1":"Major Area/Country",
    "Unnamed: 2":"Notes",
    "Unnamed: 3":"Country Code",
    "Unnamed: 4":"Type of Data",
    "Unnamed: 5":"Both sexes_1990",
    "Unnamed: 6":"Both sexes_1995",
    "Unnamed: 7":"Both sexes_2000",
    "Unnamed: 8":"Both sexes_2005",
    "Unnamed: 9":"Both sexes_2010",
    "Unnamed: 10":"Both sexes_2015",
    "Unnamed: 11":"M_1990",
    "Unnamed: 12":"M_1995",
    "Unnamed: 13":"M_2000",
    "Unnamed: 14":"M_2005",
    "Unnamed: 15":"M_2010",
    "Unnamed: 16":"M_2015",
    "Unnamed: 17":"F_1990",
    "Unnamed: 18":"F_1995",
    "Unnamed: 19":"F_2000",
    "Unnamed: 20":"F_2005",
    "Unnamed: 21":"F_2010",
    "Unnamed: 22":"F_2015"
}, inplace = True)

#Remove blank rows
table3 = table3.iloc[15:]

### Unpivot Columns Containing Year and Gender; Clean "Type of Data" Column (Table 3) 

In [50]:
#Use the melt function to unpivot the columns containing year and gender information
table3_melt = pd.melt(table3, id_vars = [
    'Sort/Order',
    'Major Area/Country',
    'Notes',
    'Country Code',
    'Type of Data'], var_name = "Sex_Year", value_name = "International Migrant Stock as Percentage of the Total Population"
)

table3_melt[["Sex", "Year"]] = table3_melt["Sex_Year"].str.split("_", expand = True)

table3_melt = table3_melt.drop(columns = ["Sex_Year"])

#replace nan with na in column "Type of Data"
table3_melt["Type of Data"] = table3_melt["Type of Data"].replace(to_replace = [np.nan], value = ["na"])

#Replace values in "Type of Data" with the descriptions in the dictionary
table3_melt["Type of Data"] = table3_melt["Type of Data"].apply(lambda x: "; ".join([data_type_dict[i] for i in x.strip().split()]))


### Use Inner Join Function to Remove World and Region Data from "Major Area/County" column (Table 3)

In [51]:
table3_joined = table3_melt.join(annex, on = "Country Code", how = "inner")

#Remove the .. from the percentage 
table3_joined["International Migrant Stock as Percentage of the Total Population"] = table3_joined["International Migrant Stock as Percentage of the Total Population"].replace(to_replace = [".."], value = [np.nan])

#set the percentage as float
table3_joined["International Migrant Stock as Percentage of the Total Population"] = table3_joined["International Migrant Stock as Percentage of the Total Population"].astype(float)

### Rename column values and rearrange columns (Table 3)

In [52]:
#Replace codes for sexes with proper labels
table3_joined["Sex"] = table3_melt["Sex"].replace(to_replace = ["Both sexes", "M", "F"], value = ["Both Sexes", "Male", "Female"])

#Clean up the order of the table & reset index 
table3_clean = table3_joined.drop(columns = ["Sort/Order", "Major Area/Country"]).reset_index(drop = True)

table3_clean= table3_clean [[
    'Major Area Sort Order',
    'Major Area Code',
    'Major Area',
    'Region Sort Order',
    'Region Code',
    'Region',
    'Country Sort Order',
    'Country Code',
    'Country or Area',
    'Year',
    'Sex',
    'International Migrant Stock as Percentage of the Total Population',
    'Developed Region',
    'Least Developed Country',
    'Sub-Saharan Africa',
    'Type of Data',
    'Notes'
]]

#Reset index 
table3_clean.reset_index(inplace = True, drop = True)

pd.set_option("display.precision", 2)

#Display cleaned table 3
table3_clean

,Major Area Sort Order,Major Area Code,Major Area,Region Sort Order,Region Code,Region,Country Sort Order,Country Code,Country or Area,Year,Sex,International Migrant Stock as Percentage of the Total Population,Developed Region,Least Developed Country,Sub-Saharan Africa,Type of Data,Notes
0,7,903,Africa,8,910,Eastern Africa,9,108,Burundi,1990,Both Sexes,5.93,No,Yes,Yes,Foreign-born Population; Refugee Population In...,NaN
1,7,903,Africa,8,910,Eastern Africa,9,108,Burundi,1995,Both Sexes,4.08,No,Yes,Yes,Foreign-born Population; Refugee Population In...,NaN
2,7,903,Africa,8,910,Eastern Africa,9,108,Burundi,2000,Both Sexes,1.86,No,Yes,Yes,Foreign-born Population; Refugee Population In...,NaN
3,7,903,Africa,8,910,Eastern Africa,9,108,Burundi,2005,Both Sexes,2.18,No,Yes,Yes,Foreign-born Population; Refugee Population In...,NaN
4,7,903,Africa,8,910,Eastern Africa,9,108,Burundi,2010,Both Sexes,2.49,No,Yes,Yes,Foreign-born Population; Refugee Population In...,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4171,238,909,Oceania,256,957,Polynesia,265,876,Wallis and Futuna Islands,1995,Female,NaN,No,No,No,Foreign-born Population,NaN
4172,238,909,Oceania,256,957,Polynesia,265,876,Wallis and Futuna Islands,2000,Female,NaN,No,No,No,Foreign-born Population,NaN
4173,238,909,Oceania,256,957,Polynesia,265,876,Wallis and Futuna Islands,2005,Female,NaN,No,No,No,Foreign-born Population,NaN
4174,238,909,Oceania,256,957,Polynesia,265,876,Wallis and Futuna Islands,2010,Female,NaN,No,No,No,Foreign-born Population,NaN


## Read Table 4 from Github

In [53]:
table4 = pd.read_excel(r"https://github.com/teresalau/inf1340-programmingfordatascience-fa22/raw/main/project%20data/UN_MigrantStockTotal_2015.xlsx", sheet_name = "Table 4")

### Remove Blank Rows & Rename Column Headers (Table 4)

In [54]:
#Remove top 14 rows as they are blank
table4_removerows = table4.iloc[15:].rename(columns = {
    "Unnamed: 0":"Sort\norder",
    "Unnamed: 1": "Major Area/Region/Country",
    "Unnamed: 2":"Notes",
    "Unnamed: 3":"Country Code",
    "Unnamed: 4":"Type of Data",
    "Unnamed: 5":"F_1990",
    "Unnamed: 6":"F_1995",
    "Unnamed: 7":"F_2000",
    "Unnamed: 8":"F_2005",
    "Unnamed: 9":"F_2010",
    "Unnamed: 10":"F_2015"
})

### Use melt to unpivot year columns (Table 4)

In [55]:
#Unpivot columns containing migrant stock percentage information as rows
table4_melt = pd.melt(table4_removerows, id_vars = ["Sort\norder",
                                              "Major Area/Region/Country",
                                              "Notes",
                                              "Country Code",
                                              "Type of Data"], var_name = "Sex_Year", value_name = "Percentage of International Migrant Stock"
                     )
#Split Sex & Year
table4_melt[["Sex", "Year"]] = table4_melt["Sex_Year"].str.split("_", expand = True)

#Remove Gender_Year column 
table4_melt = table4_melt.drop(columns = ["Sex_Year"]).replace({"F":"Female"})


### Clean Type of Data Column (Table 4)

In [56]:
#Find unique values
table4_melt["Type of Data"].unique()

#Replace nan with na in column "Type of Data" as it leads to error in the data replacement process
table4_melt["Type of Data"] = table4_melt["Type of Data"].replace(to_replace = [np.nan], value = ["na"])

#Replacing values in "Type of Data" with the descriptions in the dictionary
table4_melt["Type of Data"] = table4_melt["Type of Data"].apply(lambda x: "; ".join([data_type_dict[i] for i in x.strip().split()]))

### Clean Major Area/Region/Country by Inner Join (Join with Annex table) (Table 4)

In [57]:
table4_joined = table4_melt.join(annex, on = "Country Code", how = "inner")

### Convert percentage columns to float (Table 4)

In [58]:
#Discovered there is a stg ".." in the migrant # column, replace str with nan

table4_joined["Percentage of International Migrant Stock"] = table4_joined["Percentage of International Migrant Stock"].replace(to_replace = [".."], value = [np.nan])

#divide current number by 100
#table4_joined["Percentage of International Migrant Stock"]= table4_joined["Percentage of International Migrant Stock"]/100

table4_joined["Percentage of International Migrant Stock"] = table4_joined["Percentage of International Migrant Stock"].astype(float)

### Clean Up Duplicate Columns & Re-arrange Columns (Table 4)

In [59]:
#clean up duplicate columns as a result of the inner join

table4_clean = table4_joined.drop(columns = ['Sort\norder','Major Area/Region/Country']).reset_index(drop = True)

table4_clean = table4_clean[[
    'Major Area Sort Order',
    'Major Area Code',
    'Major Area',
    'Region Sort Order',
    'Region Code',
    'Region',
    'Country Sort Order',
    'Country or Area',
    'Country Code',
    'Year',
    'Sex',
    'Percentage of International Migrant Stock',
    'Type of Data',
    'Developed Region',
    'Least Developed Country',
    'Sub-Saharan Africa',
    'Notes'
]]

pd.set_option("display.precision", 2)

#Display cleaned table 4
table4_clean

,Major Area Sort Order,Major Area Code,Major Area,Region Sort Order,Region Code,Region,Country Sort Order,Country or Area,Country Code,Year,Sex,Percentage of International Migrant Stock,Type of Data,Developed Region,Least Developed Country,Sub-Saharan Africa,Notes
0,7,903,Africa,8,910,Eastern Africa,9,Burundi,108,1990,Female,50.99,Foreign-born Population; Refugee Population In...,No,Yes,Yes,NaN
1,7,903,Africa,8,910,Eastern Africa,9,Burundi,108,1995,Female,51.28,Foreign-born Population; Refugee Population In...,No,Yes,Yes,NaN
2,7,903,Africa,8,910,Eastern Africa,9,Burundi,108,2000,Female,51.37,Foreign-born Population; Refugee Population In...,No,Yes,Yes,NaN
3,7,903,Africa,8,910,Eastern Africa,9,Burundi,108,2005,Female,50.94,Foreign-born Population; Refugee Population In...,No,Yes,Yes,NaN
4,7,903,Africa,8,910,Eastern Africa,9,Burundi,108,2010,Female,50.77,Foreign-born Population; Refugee Population In...,No,Yes,Yes,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1387,238,909,Oceania,256,957,Polynesia,265,Wallis and Futuna Islands,876,1995,Female,48.87,Foreign-born Population,No,No,No,NaN
1388,238,909,Oceania,256,957,Polynesia,265,Wallis and Futuna Islands,876,2000,Female,49.48,Foreign-born Population,No,No,No,NaN
1389,238,909,Oceania,256,957,Polynesia,265,Wallis and Futuna Islands,876,2005,Female,49.51,Foreign-born Population,No,No,No,NaN
1390,238,909,Oceania,256,957,Polynesia,265,Wallis and Futuna Islands,876,2010,Female,49.53,Foreign-born Population,No,No,No,NaN


## Import Table 5 from Github

In [60]:
table5 = pd.read_excel(r"https://github.com/teresalau/inf1340-programmingfordatascience-fa22/raw/main/project%20data/UN_MigrantStockTotal_2015.xlsx", sheet_name = "Table 5")

### Rename column headers (Table 5)

In [61]:
#Rename columns
table5.rename(columns = {
    'Unnamed: 0': "Sort/Order",
    'Unnamed: 1':"Major Area/Country",
    'Unnamed: 2':"Notes",
    'Unnamed: 3':"Country Code",
    'Unnamed: 4':"Type of Data",
    'Unnamed: 5':"Both_1990-1995",
    'Unnamed: 6':"Both_1995-2000",
    'Unnamed: 7':"Both_2000-2005",
    'Unnamed: 8':"Both_2005-2010",
    'Unnamed: 9':"Both_2010-2015",
    'Unnamed: 10':"M_1990-1995",
    'Unnamed: 11':"M_1995-2000",
    'Unnamed: 12':"M_2000-2005",
    'Unnamed: 13':"M_2005-2010",
    'Unnamed: 14':"M_2010-2015",
    'Unnamed: 15':"F_1990-1995",
    'Unnamed: 16':"F_1995-2000",
    'Unnamed: 17':"F_2000-2005",
    'Unnamed: 18':"F_2005-2010",
    'Unnamed: 19':"F_2010-2015"
}, inplace = True)

#Remove blank rows at the top
table5 = table5.iloc[15:]

### Unpivot columns with years and sex (Table 5)

In [62]:
#Unpiovt Year Range and Sex columns 
table5_melt = pd.melt(table5, id_vars = [
    "Sort/Order","Major Area/Country","Notes","Country Code","Type of Data"], var_name = "Sex_YearRange", 
          value_name ="Annual Rate of Change of the Migrant Stock")

#Split Sex and Year Range
table5_melt[["Sex", "Year Range"]] = table5_melt["Sex_YearRange"].str.split("_",expand = True)
table5_melt = table5_melt.drop(columns = ["Sex_YearRange"])

### Use Inner Join to extract country from the "Major Area Country column" (Table 5)

In [63]:
table5_joined = table5_melt.join(annex, on = "Country Code", how = "inner")

### Clean the "Type of Data" column (Table 5)

In [64]:
#replace nan with na in column "Type of Data"
table5_joined["Type of Data"] = table5_joined["Type of Data"].replace(to_replace = [np.nan], value = ["na"])

#Replace values in "Type of Data" with the descriptions in the dictionary
table5_joined["Type of Data"] = table5_joined["Type of Data"].apply(lambda x: "; ".join([data_type_dict[i] for i in x.strip().split()]))

### Replace values in Sex column with proper label & rearrange table for readibility (Table 5)

In [65]:
#Replace codes for sexes with proper labels
table5_joined["Sex"] = table5_melt["Sex"].replace(to_replace = ["Both", "M", "F"], value = ["Both Sexes", "Male", "Female"])

#Re-arrange columns for readibility & reset index
table5_clean = table5_joined.drop(columns = ["Sort/Order", "Major Area/Country"]).reset_index(drop = True)

table5_clean= table5_clean [[
    'Major Area Sort Order',
    'Major Area Code',
    'Major Area',
    'Region Sort Order',
    'Region Code',
    'Region',
    'Country Sort Order',
    'Country Code',
    'Country or Area',
    'Year Range',
    'Sex',
    'Annual Rate of Change of the Migrant Stock',
    'Developed Region',
    'Least Developed Country',
    'Sub-Saharan Africa',
    'Type of Data',
    'Notes'
]]

#Replace ".." with nan in the annual rate of change column
table5_clean["Annual Rate of Change of the Migrant Stock"] = table5_clean["Annual Rate of Change of the Migrant Stock"].replace(to_replace = [".."], value = [np.nan])

#Change the column data type to float
table5_clean.astype({"Annual Rate of Change of the Migrant Stock": "float64"})

pd.set_option("display.precision", 2)

#Display clean table 5
table5_clean

,Major Area Sort Order,Major Area Code,Major Area,Region Sort Order,Region Code,Region,Country Sort Order,Country Code,Country or Area,Year Range,Sex,Annual Rate of Change of the Migrant Stock,Developed Region,Least Developed Country,Sub-Saharan Africa,Type of Data,Notes
0,7,903,Africa,8,910,Eastern Africa,9,108,Burundi,1990-1995,Both Sexes,-5.36,No,Yes,Yes,Foreign-born Population; Refugee Population In...,NaN
1,7,903,Africa,8,910,Eastern Africa,9,108,Burundi,1995-2000,Both Sexes,-14.15,No,Yes,Yes,Foreign-born Population; Refugee Population In...,NaN
2,7,903,Africa,8,910,Eastern Africa,9,108,Burundi,2000-2005,Both Sexes,6.38,No,Yes,Yes,Foreign-born Population; Refugee Population In...,NaN
3,7,903,Africa,8,910,Eastern Africa,9,108,Burundi,2005-2010,Both Sexes,6.16,No,Yes,Yes,Foreign-born Population; Refugee Population In...,NaN
4,7,903,Africa,8,910,Eastern Africa,9,108,Burundi,2010-2015,Both Sexes,3.96,No,Yes,Yes,Foreign-born Population; Refugee Population In...,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3475,238,909,Oceania,256,957,Polynesia,265,876,Wallis and Futuna Islands,1990-1995,Female,3.89,No,No,No,Foreign-born Population,NaN
3476,238,909,Oceania,256,957,Polynesia,265,876,Wallis and Futuna Islands,1995-2000,Female,3.88,No,No,No,Foreign-born Population,NaN
3477,238,909,Oceania,256,957,Polynesia,265,876,Wallis and Futuna Islands,2000-2005,Female,3.22,No,No,No,Foreign-born Population,NaN
3478,238,909,Oceania,256,957,Polynesia,265,876,Wallis and Futuna Islands,2005-2010,Female,3.21,No,No,No,Foreign-born Population,NaN


## Import Table 6 from Github

In [66]:
table6= pd.read_excel(r"https://github.com/teresalau/inf1340-programmingfordatascience-fa22/raw/main/project%20data/UN_MigrantStockTotal_2015.xlsx", sheet_name = "Table 6")

### Remove blank rows and add column headers (Table 6)

In [67]:
table6=table6.iloc[15:]

#Rename columns
table6.rename(columns = {
    'Unnamed: 0':"Sort/Order",
    'Unnamed: 1':"Major Area/Country",
    'Unnamed: 2':"Notes",
    'Unnamed: 3':"Country Code",
    'Unnamed: 4':"Type of Data",
    'Unnamed: 5':"MYStock_1990",
    'Unnamed: 6':"MYStock_1995",
    'Unnamed: 7':"MYStock_2000",
    'Unnamed: 8':"MYStock_2005",
    'Unnamed: 9':"MYStock_2010",
    'Unnamed: 10':"MYStock_2015",
    'Unnamed: 11':"RefugeesPercent_1990",
    'Unnamed: 12':"RefugeesPercent_1995",
    'Unnamed: 13':"RefugeesPercent_2000",
    'Unnamed: 14':"RefugeesPercent_2005",
    'Unnamed: 15':"RefugeesPercent_2010",
    'Unnamed: 16':"RefugeesPercent_2015",
    'Unnamed: 17':"ARChange_90-95",
    'Unnamed: 18':"ARChange_95-00",
    'Unnamed: 19':"ARChange_00-05",
    'Unnamed: 20':"ARChange_05-10",
    'Unnamed: 21':"ARChange_10-15"
}, inplace = True)

### Use melt to unpivot columns containing indicator & year data (Table 6)

In [68]:
table6_melt = pd.melt(table6, id_vars =[
    "Sort/Order",'Major Area/Country', 'Notes', 'Country Code','Type of Data'], 
                      var_name = "Indicator_Year", value_name = "Value"
                     )

#Split Indicator and Value into two columns
table6_melt[["Indicator", "Year"]] = table6_melt["Indicator_Year"].str.split("_",expand = True)
table6_melt = table6_melt.drop(columns = ["Indicator_Year"])


### Clean the "Type of Data" Column (Table 6)

In [69]:
#replace nan with na in column "Type of Data"
table6_melt["Type of Data"] = table6_melt["Type of Data"].replace(to_replace = [np.nan], value = ["na"])

#Replace values in "Type of Data" with the descriptions in the dictionary
table6_melt["Type of Data"] = table6_melt["Type of Data"].apply(lambda x: "; ".join([data_type_dict[i] for i in x.strip().split()]))

### Use Inner Join to seperate major area and region from country (Table 6)

In [70]:
#Inner Join with Annex Table, then reset index, and drop the duplicate columns
table6_joined = table6_melt.join(annex, on = "Country Code", how = "inner").reset_index(drop = True).drop(columns = [
    "Sort/Order", "Major Area/Country"])

#Re-arrange columns for readibility
table6_joined= table6_joined [[
    'Major Area Sort Order',
    'Major Area Code',
    'Major Area',
    'Region Sort Order',
    'Region Code',
    'Region',
    'Country Sort Order',
    'Country Code',
    'Country or Area',
    'Year',
    'Indicator',
    'Value',
    'Developed Region',
    'Least Developed Country',
    'Sub-Saharan Africa',
    'Type of Data',
    'Notes'
]]


### Create a seperate data table for estimated migrant stock at mid-year data (Table 6)

In [125]:
#Create a seperate data table for estimated migrant stock at mid-year
table6_MyStock = table6_joined[table6_joined["Indicator"] == "MYStock"].copy()

#Reset index and rename column header to indicate the variable where the values represent
#Drop the indicator column to remove redundent information
table6_MyStock = table6_MyStock.reset_index(drop = True).rename(columns = {
    "Value": "Estimated Refugee Stock at Mid-Year Both Sexes"}).drop(columns = ["Indicator"])

#Replace ".." with nan in the annual rate of change column
table6_MyStock["Estimated Refugee Stock at Mid-Year Both Sexes"] = table6_MyStock["Estimated Refugee Stock at Mid-Year Both Sexes"].replace(to_replace = [".."], value = [np.nan])

table6_MyStock.astype({"Estimated Refugee Stock at Mid-Year Both Sexes": "Int64"})

#Display 2 decimal place
pd.set_option("display.precision", 2)

#Display cleaned table 6 - Mid-Year Migrant Stock
table6_MyStock

,Major Area Sort Order,Major Area Code,Major Area,Region Sort Order,Region Code,Region,Country Sort Order,Country Code,Country or Area,Year,Estimated Refugee Stock at Mid-Year Both Sexes,Developed Region,Least Developed Country,Sub-Saharan Africa,Type of Data,Notes
0,7,903,Africa,8,910,Eastern Africa,9,108,Burundi,1990,267929,No,Yes,Yes,Foreign-born Population; Refugee Population In...,NaN
1,7,903,Africa,8,910,Eastern Africa,9,108,Burundi,1995,173017,No,Yes,Yes,Foreign-born Population; Refugee Population In...,NaN
2,7,903,Africa,8,910,Eastern Africa,9,108,Burundi,2000,27136,No,Yes,Yes,Foreign-born Population; Refugee Population In...,NaN
3,7,903,Africa,8,910,Eastern Africa,9,108,Burundi,2005,20681,No,Yes,Yes,Foreign-born Population; Refugee Population In...,NaN
4,7,903,Africa,8,910,Eastern Africa,9,108,Burundi,2010,29365,No,Yes,Yes,Foreign-born Population; Refugee Population In...,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1387,238,909,Oceania,256,957,Polynesia,265,876,Wallis and Futuna Islands,1995,0,No,No,No,Foreign-born Population,NaN
1388,238,909,Oceania,256,957,Polynesia,265,876,Wallis and Futuna Islands,2000,0,No,No,No,Foreign-born Population,NaN
1389,238,909,Oceania,256,957,Polynesia,265,876,Wallis and Futuna Islands,2005,0,No,No,No,Foreign-born Population,NaN
1390,238,909,Oceania,256,957,Polynesia,265,876,Wallis and Futuna Islands,2010,0,No,No,No,Foreign-born Population,NaN


### Create a seperate data table for refugees as a percentage of the international migrant stock (Table 6)

In [124]:
#Create a seperate data table for refugees as a percentage of the international migrant stock
table6_RP = table6_joined[table6_joined["Indicator"] == "RefugeesPercent"].copy()

#Remove the .. from the value column
table6_RP["Value"] = table6_RP["Value"].replace(to_replace = [".."], value = [np.nan])

#Change type to float
table6_RP["Value"] = table6_RP["Value"].astype(float)

#Rename columns header "Value" with "Percentage" to indicate the type of data
#Drop column "Indicator" & reset index
table6_RP = table6_RP.rename(columns = {
    "Value":"Refugees as a Percentage of the International Migrant Stock"}).drop(columns = [
    "Indicator"]).reset_index(drop = True)

#Display cleaned table 6 - Refugee as a percentage of international migrant stock
table6_RP

,Major Area Sort Order,Major Area Code,Major Area,Region Sort Order,Region Code,Region,Country Sort Order,Country Code,Country or Area,Year,Refugees as a Percentage of the International Migrant Stock,Developed Region,Least Developed Country,Sub-Saharan Africa,Type of Data,Notes
0,7,903,Africa,8,910,Eastern Africa,9,108,Burundi,1990,80,No,Yes,Yes,Foreign-born Population; Refugee Population In...,NaN
1,7,903,Africa,8,910,Eastern Africa,9,108,Burundi,1995,68,No,Yes,Yes,Foreign-born Population; Refugee Population In...,NaN
2,7,903,Africa,8,910,Eastern Africa,9,108,Burundi,2000,22,No,Yes,Yes,Foreign-born Population; Refugee Population In...,NaN
3,7,903,Africa,8,910,Eastern Africa,9,108,Burundi,2005,12,No,Yes,Yes,Foreign-born Population; Refugee Population In...,NaN
4,7,903,Africa,8,910,Eastern Africa,9,108,Burundi,2010,12,No,Yes,Yes,Foreign-born Population; Refugee Population In...,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1387,238,909,Oceania,256,957,Polynesia,265,876,Wallis and Futuna Islands,1995,0,No,No,No,Foreign-born Population,NaN
1388,238,909,Oceania,256,957,Polynesia,265,876,Wallis and Futuna Islands,2000,0,No,No,No,Foreign-born Population,NaN
1389,238,909,Oceania,256,957,Polynesia,265,876,Wallis and Futuna Islands,2005,0,No,No,No,Foreign-born Population,NaN
1390,238,909,Oceania,256,957,Polynesia,265,876,Wallis and Futuna Islands,2010,0,No,No,No,Foreign-born Population,NaN


### Create a seperate data table for annual rate of change of refugee stock (Table 6)

In [126]:
#Create a seperate table for annual rate of change data

table6_AR = table6_joined[table6_joined["Indicator"] == "ARChange"].copy()

table6_AR = table6_AR.replace({
 "90-95":"1990-1995","95-00":"1995-2000","00-05":"2000-2005","05-10":"2005-2010","10-15":"2010-2015"
})

#Remove the .. from the value column
table6_AR["Value"] = table6_AR["Value"].replace(to_replace = [".."], value = [np.nan])

#Rename columns header "Value" with indicator name to indicate the type of data
table6_AR["Value"] = table6_AR.rename(columns = {"Value":"Annual Rate of Change of the Refugee Stock"}, inplace = True)

#Change type to float
table6_AR["Annual Rate of Change of the Refugee Stock"] = table6_AR["Annual Rate of Change of the Refugee Stock"].astype(float)

#Drop the "Indicator" column & reset index
table6_AR = table6_AR.drop(columns = ["Indicator"]).reset_index(drop = True)

pd.set_option("display.precision", 2)

#Display cleaned table 6 - Annual Rate of Change of Refugee Stock
table6_AR

,Major Area Sort Order,Major Area Code,Major Area,Region Sort Order,Region Code,Region,Country Sort Order,Country Code,Country or Area,Year,Annual Rate of Change of the Refugee Stock,Developed Region,Least Developed Country,Sub-Saharan Africa,Type of Data,Notes,Value
0,7,903,Africa,8,910,Eastern Africa,9,108,Burundi,1990-1995,-3,No,Yes,Yes,Foreign-born Population; Refugee Population In...,NaN,None
1,7,903,Africa,8,910,Eastern Africa,9,108,Burundi,1995-2000,-23,No,Yes,Yes,Foreign-born Population; Refugee Population In...,NaN,None
2,7,903,Africa,8,910,Eastern Africa,9,108,Burundi,2000-2005,-12,No,Yes,Yes,Foreign-born Population; Refugee Population In...,NaN,None
3,7,903,Africa,8,910,Eastern Africa,9,108,Burundi,2005-2010,1,No,Yes,Yes,Foreign-born Population; Refugee Population In...,NaN,None
4,7,903,Africa,8,910,Eastern Africa,9,108,Burundi,2010-2015,6,No,Yes,Yes,Foreign-born Population; Refugee Population In...,NaN,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1155,238,909,Oceania,256,957,Polynesia,265,876,Wallis and Futuna Islands,1990-1995,NaN,No,No,No,Foreign-born Population,NaN,None
1156,238,909,Oceania,256,957,Polynesia,265,876,Wallis and Futuna Islands,1995-2000,NaN,No,No,No,Foreign-born Population,NaN,None
1157,238,909,Oceania,256,957,Polynesia,265,876,Wallis and Futuna Islands,2000-2005,NaN,No,No,No,Foreign-born Population,NaN,None
1158,238,909,Oceania,256,957,Polynesia,265,876,Wallis and Futuna Islands,2005-2010,NaN,No,No,No,Foreign-born Population,NaN,None


# Part II: Data Visualization

In [74]:
#Display Options

pd.set_option('display.float_format', '{:.0f}'.format)

## Table 1 - Migrant Stock

In [75]:
#Summary Statistics Table- International Migrant Stock (Both Sexes; 1990-2015)

table1_cleanBS = table1_clean[table1_clean["Sex"] == "Both Sexes"]
table1_cleanBSPivot = table1_clean[[
    "Country or Area",
    "Year",
    "Migrant Stock"
]]

table1_cleanBSPivot = pd.pivot_table(table1_cleanBSPivot, values = "Migrant Stock",
                                   index = ["Country or Area"], columns = ["Year"])

table1_cleanBSPivot.describe()

Year,1990,1995,2000,2005,2010,2015
count,228,228,228,229,232,232
mean,446091,470181,504980,556824,637110,700288
std,1357674,1532156,1782151,1980496,2228737,2365115
min,180,175,145,122,103,94
25%,13631,15100,16380,16995,17573,18720
50%,67849,74335,83764,85111,85388,99817
75%,312046,343952,362371,392697,385624,469514
max,15500684,18967369,23209369,26172195,29455762,31084735


In [76]:
#Top three countries by the highest number of migrant stock in 2015

table1_clean2015 = table1_clean[(table1_clean["Year"] == "2015") & (table1_clean["Sex"] == "Both Sexes")]
table1_clean2015.nlargest(3, "Migrant Stock")

,Major Area Sort Order,Major Area Code,Major Area,Region Sort Order,Region Code,Region,Country Sort Order,Country Code,Country or Area,Year,Sex,Migrant Stock,Type of Data,Developed Region,Least Developed Country,Sub-Saharan Africa,Notes
3749,232,905,Northern America,232,905,Northern America,237,840,United States of America,2015,Both Sexes,46627102,Foreign-born Population,Yes,No,No,NaN
2705,127,908,Europe,170,926,Western Europe,174,276,Germany,2015,Both Sexes,12005690,Foreign-born Population,Yes,No,No,NaN
2075,127,908,Europe,128,923,Eastern Europe,136,643,Russian Federation,2015,Both Sexes,11643276,Foreign-born Population,Yes,No,No,NaN


In [77]:
#Countries with the least number of migrant stock in 2015

table1_clean2015.nsmallest(3, "Migrant Stock")

,Major Area Sort Order,Major Area Code,Major Area,Region Sort Order,Region Code,Region,Country Sort Order,Country Code,Country or Area,Year,Sex,Migrant Stock,Type of Data,Developed Region,Least Developed Country,Sub-Saharan Africa,Notes
4145,238,909,Oceania,256,957,Polynesia,264,798,Tuvalu,2015,Both Sexes,141,Foreign Citizen,No,Yes,No,NaN
4109,238,909,Oceania,256,957,Polynesia,262,772,Tokelau,2015,Both Sexes,487,Foreign-born Population,No,No,No,NaN
4073,238,909,Oceania,256,957,Polynesia,260,570,Niue,2015,Both Sexes,557,Foreign-born Population,No,No,No,NaN


In [102]:
#Line graph of the international migrant stock by sex from 1990 to 2015

import plotly.io as pio
import plotly.express as px

pio.templates.default = "plotly_white"

MigrantStockMF= table1_clean.groupby(["Year", "Sex"]).sum("MigrantStock").reset_index()
MigrantStockMF = MigrantStockMF[(MigrantStockMF["Sex"] == "Male") | (MigrantStockMF["Sex"] == "Female") ]

fig1 = px.line(MigrantStockMF, x="Year", y = "Migrant Stock", title = "World's International Migrant Stock by Sex (1990-2015)", 
               color = "Sex", color_discrete_sequence=px.colors.qualitative.Set2)
fig1.show()

In [104]:
#Line graph of international migrant stock by major area (i.e. continent)

MSArea = table1_clean.groupby(["Year", "Sex", "Major Area"]).sum("MigrantStock").reset_index()
MSArea = MSArea[MSArea["Sex"] == "Both Sexes"]

fig2 = px.line(MSArea, x = "Year", y = "Migrant Stock", title = "International Migrant Stock at Mid-Year by Major Area (1990-2015)",
              color = "Major Area", markers = True, color_discrete_sequence=px.colors.qualitative.Safe)

fig2.show()

In [127]:
#Line graph of international migrant stock at mid-year by Asian region (1990-2015)

MSAsianRegion = table1_clean.groupby(["Year", "Sex", "Major Area", "Region"]).sum("Migrant Stock").reset_index()
MSAsianRegion = MSAsianRegion[(MSAsianRegion["Major Area"] == "Asia") & (MSAsianRegion["Sex"] == "Both Sexes")].reset_index()

fig3 = px.line(MSAsianRegion, x = "Year", y = "Migrant Stock", color = "Region",
              title = "International Migrant Stock at Mid-Year by Asian Region (1990-2015)",
              markers = True, color_discrete_sequence=px.colors.qualitative.Pastel)

fig3.show()

## Table 2 - Total Population Analysis

In [81]:
#Summary Statistics Table- Total Population (Both Sexes; 1990-2015)

table2_cleanBS = table2_clean[table2_clean["Sex"] == "Both Sexes"]

table2_cleanBSPivot = table2_cleanBS[[
    "Country or Area",
    "Year",
    "Total Population"
]]

table2_cleanBSPivot = pd.pivot_table(table2_cleanBSPivot, values = "Total Population",
                                    index = ["Country or Area"], columns = ["Year"])

table2_cleanBSPivot.describe()

Year,1990,1995,2000,2005,2010,2015
count,232,232,232,232,232,232
mean,22799290,24629170,26313305,28004016,29769503,31577979
std,98307383,105897506,112131477,118041381,123862854,129491149
min,770,781,787,798,799,800
25%,259515,291295,324319,388264,394343,413109
50%,3742168,4033644,4130898,4478624,4790458,5300646
75%,11987717,13995678,15313993,16726970,17794213,18754641
max,1154605773,1227841281,1269974572,1305600630,1340968737,1376048943


In [82]:
#Countries with highest population in 2015

table2_clean2015 = table2_clean[(table2_clean["Year"] == "2015") & (table2_clean["Sex"] == "Both Sexes")]
table2_clean2015.nlargest(3, "Total Population")

,Major Area Sort Order,Major Area Code,Major Area,Region Sort Order,Region Code,Region,Country Sort Order,Country Code,Country or Area,Year,Sex,Total Population,Developed Region,Least Developed Country,Sub-Saharan Africa,Notes
1139,71,935,Asia,78,906,Eastern Asia,79,156,China,2015,Both Sexes,1376048943,No,No,No,(5)
1517,71,935,Asia,98,5501,Southern Asia,102,356,India,2015,Both Sexes,1311050527,No,No,No,NaN
3749,232,905,Northern America,232,905,Northern America,237,840,United States of America,2015,Both Sexes,321773631,Yes,No,No,NaN


In [83]:
#Countries with lowest population in 2015

table2_clean2015.nsmallest(3, "Total Population")

,Major Area Sort Order,Major Area Code,Major Area,Region Sort Order,Region Code,Region,Country Sort Order,Country Code,Country or Area,Year,Sex,Total Population,Developed Region,Least Developed Country,Sub-Saharan Africa,Notes
2471,127,908,Europe,153,925,Southern Europe,160,336,Holy See,2015,Both Sexes,800,Yes,No,No,(17)
4109,238,909,Oceania,256,957,Polynesia,262,772,Tokelau,2015,Both Sexes,1250,No,No,No,NaN
4073,238,909,Oceania,256,957,Polynesia,260,570,Niue,2015,Both Sexes,1610,No,No,No,NaN


In [128]:
#Line graph of total population by sex from 1990 to 2015

PopulationMFSum= table2_clean.groupby(["Year", "Sex"]).sum("Total Population").reset_index()
PopulationMFSum = PopulationMFSum[PopulationMFSum["Sex"] != "Both Sexes"]

fig4 = px.line(PopulationMFSum, x = "Year", y = "Total Population", color = "Sex",
              title = "World's Total Population at Mid-Year by Sex (1990-2015)", 
              markers = True, color_discrete_sequence=px.colors.qualitative.Set2)
fig4.show()

## Table 3 - International Migrant Stock as Percentage of Total Population

In [131]:
#Small multiples of international migrant stock as percentage of total population by major area, across 1990 to 2015

fig5 = px.box(table3_clean, x = "International Migrant Stock as Percentage of the Total Population",
            y = "Major Area",
            facet_col = "Year",
            facet_col_wrap = 2,
            facet_row_spacing = 0.03,
            width = 1200,
            height = 800, title = "International Migrant Stock as Percentage of Total Population by Major Area (Both Sexes, 1990 to 2015)")

fig5.show()

In [86]:
#Data points with the highest percentage of international migrant stock as part of the total population

table3_clean.nlargest(5,"International Migrant Stock as Percentage of the Total Population")

,Major Area Sort Order,Major Area Code,Major Area,Region Sort Order,Region Code,Region,Country Sort Order,Country Code,Country or Area,Year,Sex,International Migrant Stock as Percentage of the Total Population,Developed Region,Least Developed Country,Sub-Saharan Africa,Type of Data,Notes
2466,127,908,Europe,153,925,Southern Europe,160,336,Holy See,1990,Both Sexes,100,Yes,No,No,No Data on International Migrant,(17)
2468,127,908,Europe,153,925,Southern Europe,160,336,Holy See,2000,Both Sexes,100,Yes,No,No,No Data on International Migrant,(17)
2470,127,908,Europe,153,925,Southern Europe,160,336,Holy See,2010,Both Sexes,100,Yes,No,No,No Data on International Migrant,(17)
2471,127,908,Europe,153,925,Southern Europe,160,336,Holy See,2015,Both Sexes,100,Yes,No,No,No Data on International Migrant,(17)
2467,127,908,Europe,153,925,Southern Europe,160,336,Holy See,1995,Both Sexes,100,Yes,No,No,No Data on International Migrant,(17)


## Table 4 - Female Migrant as Percentage of Migrant Stock

In [141]:
#Boxplot of female migrant as percentage of international migrant stock by developed & non-developed region (1990-2015)

legend_order = {"Year": ["1990", "1995","2000", "2005", "2010", "2015"],
               "Developed Region": ["Yes", "No"]}

fig6 = px.box(table4_clean, x = "Percentage of International Migrant Stock", y = "Year",
             color = "Developed Region", color_discrete_sequence=px.colors.qualitative.Safe,
              category_orders = legend_order,
              title = "Female Migrant as Percentage of International Migrant Stock by Developed & Non-developed Region (1990-2015)")

fig6.show()

## Table 5 - Annual Rate of Change of Migrant Stock at Mid-Year

In [136]:
#Boxplot of annual rate of change of migrant stock at mid-year

table5_BS = table5_clean[table5_clean["Sex"] == "Both Sexes"]

legend_order = {"Year Range": ["1990-1995", "1995-2000","2000-2005", "2005-2010", "2010-2015"],
               "Developed Region": ["Yes", "No"]}


fig7 = px.box(table5_BS, x = "Annual Rate of Change of the Migrant Stock",
            y = "Year Range",
            color = "Developed Region",
            width = 870,
            height = 400, title = "Annual Rate of Change in Migrant Stock by Developed & Non-Developed Region (Both Sexes, 1990-2015)",
      hover_data = ["Annual Rate of Change of the Migrant Stock", "Country or Area"],
       category_orders = legend_order, color_discrete_sequence=px.colors.qualitative.Safe
      )

fig7.show()

In [89]:
#Find the top five countries with the highest annual rate of change in migrant stock

table5_BS.nlargest(5, "Annual Rate of Change of the Migrant Stock")

,Major Area Sort Order,Major Area Code,Major Area,Region Sort Order,Region Code,Region,Country Sort Order,Country Code,Country or Area,Year Range,Sex,Annual Rate of Change of the Migrant Stock,Developed Region,Least Developed Country,Sub-Saharan Africa,Type of Data,Notes
2145,127,908,Europe,153,925,Southern Europe,166,688,Serbia,1990-1995,Both Sexes,37,Yes,No,No,Foreign-born Population,(18)
1219,71,935,Asia,98,5501,Southern Asia,99,4,Afghanistan,2010-2015,Both Sexes,26,No,Yes,No,Foreign-born Population,NaN
347,7,903,Africa,29,911,Middle Africa,33,148,Chad,2000-2005,Both Sexes,24,No,Yes,Yes,Foreign-born Population; Refugee Population In...,NaN
229,7,903,Africa,8,910,Eastern Africa,24,728,South Sudan,2010-2015,Both Sexes,23,No,Yes,Yes,Foreign-born Population; Refugee Population In...,NaN
1429,71,935,Asia,108,922,Western Asia,114,368,Iraq,2010-2015,Both Sexes,22,No,No,No,Foreign Citizen; Refugee Population Included i...,NaN


## Table 6.1 - Estimated Refugee Stock Mid-Year

In [90]:
# Summary Statistics - Estimated Refugee Stock at Mid-Year (Both Sexes)

table6_cleanBSPivot = table6_MyStock[[
    "Country or Area",
    "Year",
    "Estimated Refugee Stock at Mid-Year Both Sexes"
]]

table6_cleanBSPivot = pd.pivot_table(table6_cleanBSPivot, values = "Estimated Refugee Stock at Mid-Year Both Sexes",
                                   index = ["Country or Area"], columns = ["Year"])
table6_cleanBSPivot.describe()

Year,1990,1995,2000,2005,2010,2015
count,229,229,229,230,232,232
mean,82256,77964,69117,57725,66253,84386
std,348465,250220,246876,211034,270727,305259
min,0,0,0,0,0,0
25%,0,0,0,0,0,0
50%,30,626,528,552,594,640
75%,15605,22314,17685,11150,15117,18970
max,3512201,2071988,2001466,1972086,2372937,2751479


In [91]:
#Top 5 countries with the highest number of refugees in 2015

table6_MyStock2015 = table6_MyStock[table6_MyStock["Year"] == "2015"]

table6_MyStock2015.nlargest(5, "Estimated Refugee Stock at Mid-Year Both Sexes")

,Major Area Sort Order,Major Area Code,Major Area,Region Sort Order,Region Code,Region,Country Sort Order,Country Code,Country or Area,Year,Estimated Refugee Stock at Mid-Year Both Sexes,Developed Region,Least Developed Country,Sub-Saharan Africa,Type of Data,Notes
587,71,935,Asia,108,922,Western Asia,116,400,Jordan,2015,2751479,No,No,No,Foreign Citizen; Refugee Population Included i...,NaN
623,71,935,Asia,108,922,Western Asia,119,275,State of Palestine,2015,2051096,No,No,No,Foreign-born Population,(12)
599,71,935,Asia,108,922,Western Asia,118,422,Lebanon,2015,1617179,No,No,No,Foreign-born Population; Refugee Population In...,NaN
533,71,935,Asia,98,5501,Southern Asia,106,586,Pakistan,2015,1610355,No,No,No,Foreign-born Population; Refugee Population In...,NaN
635,71,935,Asia,108,922,Western Asia,124,792,Turkey,2015,1587374,No,No,No,Foreign-born Population; Refugee Population In...,NaN


In [142]:
#Line graph of estimated refugee stock by major area (1990-2015)

table6_RefugeeMajorArea = table6_MyStock.groupby(["Year", "Major Area"]).sum("Estimated Refugee Stock at Mid-Year Both Sexes").reset_index()

fig8 = px.line(table6_RefugeeMajorArea, x = "Year", y = "Estimated Refugee Stock at Mid-Year Both Sexes",
              color = "Major Area", title = "Estimated Refugee Stock by Major Area (1990-2015)",
            markers = True, color_discrete_sequence=px.colors.qualitative.Safe)
fig8.show()

In [157]:
#Line graph of estimated refugee stock by Asian region (1990-2015)

table6_MyStockAsia = table6_MyStock[table6_MyStock["Major Area"] == "Asia"].groupby(["Year", "Region"]).sum("Estimated Refugee Stock at Mid-Year Both Sexes").reset_index()
table6_MyStockAsia
fig9 = px.line(table6_MyStockAsia, x = "Year", y = "Estimated Refugee Stock at Mid-Year Both Sexes",
              color = "Region", title = "Estimated Refugee Stock by Asian Region (1990-2015)",
              markers = True, color_discrete_sequence=px.colors.qualitative.Pastel)

fig9.show()

## Table 6.2 - Refugee as Percentage of International Migrant Stock

In [149]:
#Boxplot of refugees as percentage of international migrant stock by developed & non-developed (1990-2015)
#Without removing outlier 

legend_order = {
    "Year": ["1990", "1995", "2000", "2005", "2010", "2015"],
    "Developed Region" : ["Yes", "No"]
}

fig10 = px.box(table6_RP, y = "Year",
            x = "Refugees as a Percentage of the International Migrant Stock",
            color = "Developed Region",
            width = 900,
            height = 400, title = "Refugee as Percentage of International Migrant Stock by Developed & Non-developed Region (1990-2015)",
            category_orders = legend_order,
          hover_data = ["Refugees as a Percentage of the International Migrant Stock", "Country or Area"],
          color_discrete_sequence=px.colors.qualitative.Safe)

fig10.show()

In [150]:
#Boxplot of refugee as percentage of international migrant stock by developed & non-developed region (1990-2015)
#Removed outlier (State of Palestine)

legend_order = {
    "Year": ["1990", "1995", "2000", "2005", "2010", "2015"],
    "Developed Region" : ["Yes", "No"]
}

RefugeeMajorArea_Remove = table6_RP[table6_RP["Country or Area"] != "State of Palestine"]

fig11 = px.box(RefugeeMajorArea_Remove, y = "Year",
            x = "Refugees as a Percentage of the International Migrant Stock",
            color = "Developed Region",
            width = 900,
            height = 400, title = "Refugee as Percentage of International Migrant Stock by Developed & Non-developed Region (1990-2015)",
          category_orders = legend_order ,
          hover_data = ["Refugees as a Percentage of the International Migrant Stock", "Country or Area"],
        color_discrete_sequence=px.colors.qualitative.Safe)

fig11.show()

## Table 6.3 - Annual Rate of Change of Refugee Stock

In [153]:
#Boxplot of annual rate of change of refugee stock by developed & non-developed region (1990-2015)


legend_order = {
    "Year": ["1990-1995", "1995-2000","2000-2005", "2005-2010", "2010-2015"],
    "Developed Region" : ["Yes", "No"]}

fig11 = px.box(table6_AR, x = "Annual Rate of Change of the Refugee Stock",
            y = "Year",
            color = "Developed Region",
            width = 900,
            height = 400, title = "Annual Rate of Change in Refugee Stock by Developed & Non-Developed Region (1990-2015)",
            hover_data = ["Annual Rate of Change of the Refugee Stock", "Country or Area"],
      category_orders = legend_order,
              color_discrete_sequence=px.colors.qualitative.Safe)

fig11.show()